<a href="https://colab.research.google.com/github/jihun0423/Dacon-Hansol-img-classification/blob/main/TTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm
from sklearn import metrics


import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
device

device(type='cuda')

In [5]:

CFG = {
    'IMG_SIZE':224,
    'EPOCHS': 10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':128,
    'SEED': 41
}

In [11]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [41]:
base_dir = '/content/gdrive/MyDrive/open (2)/'
train_folder = glob.glob(base_dir + 'train/*')
train_img_list = glob.glob(base_dir + 'train/*/*')
df = pd.DataFrame(columns=['path', 'label'])
df['path'] = train_img_list
df['label'] = df['path'].apply(lambda x : str(x).split('/')[-2])

In [42]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [ ]:
plus_folder = glob.glob(base_dir + 'plus/*')

plus_path = []
for folder in plus_folder:
    tmp = glob.glob(folder + '/*')
    plus_path += tmp

In [ ]:
plus_df = pd.DataFrame(plus_path, columns=['img_path'])
plus_df['label'] = plus_df['img_path'].apply(lambda x: x.split('/')[-2])

In [ ]:
plus_df['label']=plus_df['label'].astype(int)

In [ ]:
df = pd.concat([df,plus_df])

In [43]:
df['label'].value_counts()

18    1405
10     595
1      307
3      210
15     162
2      145
11     142
7      130
6       99
9       57
5       54
17      51
14      27
12      22
13      17
4       14
0       12
16       5
8        3
Name: label, dtype: int64

In [44]:
df[df['label']==8]

,path,label
971,/content/gdrive/MyDrive/open (2)/train/반점/...,8
972,/content/gdrive/MyDrive/open (2)/train/반점/...,8
973,/content/gdrive/MyDrive/open (2)/train/반점/...,8


In [46]:
train=df

In [52]:

import os
for i in range(0,19):
  os.mkdir("/content/gdrive/MyDrive/open (2)/aug/"+f'{i}')

In [61]:
num=0
two_list = [17,5,9]
for category in two_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(2):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(),
                             A.RandomBrightnessContrast(brightness_limit=0.2), A.Resize(300,300)]) # A.VerticalFlip()
            image_aug = aug(image=image)['image']
            cv2.imwrite(f'/content/gdrive/MyDrive/open (2)/aug/{category}/'+str(num)+name,image_aug)
            # cv2.imwrite('train_300_2/' + str(i) + str(category) + name, image_aug)
            num=num+1

100%|██████████| 57/57 [00:03<00:00, 18.94it/s]


In [62]:
three_list = [0,4,13,12,14]
for category in three_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(2):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(),
                             A.RandomBrightnessContrast(brightness_limit=0.2), A.Resize(300,300)]) # A.VerticalFlip()
            image_aug = aug(image=image)['image']
            cv2.imwrite(f'/content/gdrive/MyDrive/open (2)/aug/{category}/'+str(num)+name,image_aug)
            # cv2.imwrite('train_300_2/' + str(i) + str(category) + name, image_aug)
            num=num+1


five_list = [3,16]
for category in five_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(2):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(),
                             A.RandomBrightnessContrast(brightness_limit=0.2), A.Resize(300,300)]) # A.VerticalFlip()
            image_aug = aug(image=image)['image']
            cv2.imwrite(f'/content/gdrive/MyDrive/open (2)/aug/{category}/'+str(num)+name,image_aug)
            # cv2.imwrite('train_300_2/' + str(i) + str(category) + name, image_aug)
            num=num+1

100%|██████████| 5/5 [00:07<00:00,  1.50s/it]


In [63]:
aug_folder = glob.glob(base_dir + 'aug/*')

aug_path = []
for folder in aug_folder:
    tmp = glob.glob(folder + '/*')
    aug_path += tmp

In [64]:
aug_df = pd.DataFrame(aug_path, columns=['path'])
aug_df['label'] = aug_df['path'].apply(lambda x: x.split('/')[-2])

In [65]:
aug_df['label']=aug_df['label'].astype(int)

In [66]:
aug_df

,path,label
0,/content/gdrive/MyDrive/open (2)/aug/0/32411.png,0
1,/content/gdrive/MyDrive/open (2)/aug/0/32511.png,0
2,/content/gdrive/MyDrive/open (2)/aug/0/3261.png,0
3,/content/gdrive/MyDrive/open (2)/aug/0/3271.png,0
4,/content/gdrive/MyDrive/open (2)/aug/0/3280.png,0
...,...,...
933,/content/gdrive/MyDrive/open (2)/aug/17/979.png,17
934,/content/gdrive/MyDrive/open (2)/aug/17/985.png,17
935,/content/gdrive/MyDrive/open (2)/aug/17/995.png,17
936,/content/gdrive/MyDrive/open (2)/aug/17/1006.png,17


In [68]:
train = pd.concat([train,aug_df])

In [69]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
train['label'] = train['label'].astype('str')

In [83]:
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB1
from tensorflow.keras.callbacks import *


skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
#kf = KFold(n_splits=10, random_state=42,shuffle=True)
result = 0
result2 = 0
for train_idx,valid_idx in skf.split(train,train['label']):
    x_train = train.iloc[train_idx]
    x_valid = train.iloc[valid_idx]

    es = EarlyStopping(patience = 4,restore_best_weights=True)
    rl = ReduceLROnPlateau(patience = 3,verbose=1)

    idg = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)   ## Test-Time Augmentation (TTA) 수직,수평 뒤집기
    idg2 = ImageDataGenerator()

    train_generator = idg.flow_from_dataframe(x_train, x_col = 'path',
                                              y_col = 'label',batch_size=32,target_size=(300,300)) # aug Plus
    valid_generator = idg2.flow_from_dataframe(x_valid, x_col = 'path',
                                               y_col = 'label',batch_size=32,target_size=(300,300)) # aug No
    test_generator = idg.flow_from_dataframe(test, x_col = 'path',y_col = None, batch_size=32,
                                             class_mode = None, shuffle=False,target_size=(300,300))
    test_generator2 = idg2.flow_from_dataframe(test, x_col = 'path',y_col = None, batch_size=32,
                                               class_mode = None, shuffle=False,target_size=(300,300))

    # 모델 선언
    model = Sequential()
    model.add(EfficientNetB4(include_top=False, pooling='avg'))
    model.add(Dense(19, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')
    model.fit(train_generator, validation_data=valid_generator, epochs=50, callbacks=[es,rl])


    # 예측 : TTA 30%, 일반추론 70%씩 반영
    result += model.predict(test_generator)/10 # 30%
    result2 += model.predict(test_generator2)/10 # 70%

final_result = 0.3*result + 0.7*result2

Found 3955 validated image filenames belonging to 19 classes.
Found 440 validated image filenames belonging to 18 classes.
Found 792 validated image filenames.
Found 792 validated image filenames.
71686520/71686520 [==============================] - 4s 0us/step
Epoch 1/50
124/124 [==============================] - ETA: 0s - loss: 1.1492 - acc: 0.6506 

InvalidArgumentError: ignored

In [78]:
test_img_list = glob.glob(base_dir + 'test/*/*')

In [79]:
test = pd.DataFrame(columns=['path'])
test['path'] = test_img_list

In [80]:
test_img_list = glob.glob(base_dir + 'test/*/*')

In [81]:
test = pd.DataFrame(columns=['path'])
test['path'] = test_img_list